In [1]:
model = None

## Imports 

In [2]:
# imports
import numpy as np
from urllib.parse import unquote
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential      
from keras.layers import Dense           
from keras.layers import LSTM            
from keras.layers.embeddings import Embedding 
from keras.preprocessing import sequence
from sklearn import preprocessing
from flask import Flask, request 
np.random.seed(7)

Using TensorFlow backend.


## User defined functions

In [3]:
# This function is used to normalise the values. 
def normalize(arr):
    return [(float(i)-min(arr))/(max(arr)-min(arr)) for i in arr]

# The array is formatted and the class labels are extracted from the array.

def formatArray(arr):
    last = arr[len(arr)-1]
    content = last.split("//")
    arr[len(arr)-1] = content[0]
    return arr,int(content[1])

# 
def readFile(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    varrr = []
    classLabel = []
    for i in content:
        str_v = i.split(",")
        str_v,label = formatArray(str_v)
        str_v = [int(el) for el in str_v]
        str_v = normalize(str_v)
        vv=str(str_v).strip('[]')
        vv=vv.replace(',','')
        varrr.append(vv)
        classLabel.append(label)
    return varrr,classLabel
def readData():
    df = pd.DataFrame()
    fname1 = "lr.txt"
    fname2 = "td.txt"
    classLabel = []
    varrr = []
    var,clabel = readFile(fname1)
    varrr+=var
    classLabel+=clabel
    var,clabel = readFile(fname2)
    varrr+=var
    classLabel+=clabel
    classLabel  = np.array(classLabel)
    df['sequence'] = pd.Series(varrr).str.join('')
    df['sequence'] = df['sequence'].apply(lambda x: [float(e) for e in x.split()])
    train_size = int(len(df) * (1))
    X_train = df['sequence'].values[:train_size]
    return (X_train,classLabel),(X_train,classLabel)

In [4]:
# lstm model is trained and the model object is used to predict the class for the test data.
def load_model():
    (X_train, y_train) , (X_test, y_test) = readData()
    max_review_length = 100
    X_train = sequence.pad_sequences(X_train, maxlen=max_review_length,dtype='float32')
    X_test = sequence.pad_sequences(X_test, maxlen=max_review_length,dtype='float32')
    print(X_train)
    embedding_vecor_length = 32
    global model
    model = Sequential()
    model.add(Embedding(5000, embedding_vecor_length, input_length=100))
    model.add(LSTM(50))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=50)

    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100)) 

In [ ]:
load_model()

[[0.         0.         0.         ... 0.         0.08333334 0.08333334]
 [0.         0.         0.         ... 0.03571429 0.03571429 0.03571429]
 [0.         0.         0.         ... 0.03703704 0.         0.01851852]
 ...
 [0.         0.         0.         ... 0.9512195  0.9634146  0.9390244 ]
 [0.         0.         0.         ... 0.9746835  0.96202534 0.93670887]
 [0.         0.         0.         ... 0.92941177 0.9529412  0.9411765 ]]
Epoch 1/10
598/598 [==============================] - 2s 3ms/step - loss: 0.6925 - acc: 0.5284
Epoch 2/10
598/598 [==============================] - 1s 2ms/step - loss: 0.6875 - acc: 0.5602
Epoch 3/10
598/598 [==============================] - 1s 2ms/step - loss: 0.6730 - acc: 0.5769
Epoch 4/10
598/598 [==============================] - 1s 1ms/step - loss: 0.6025 - acc: 0.7542
Epoch 5/10
598/598 [==============================] - 1s 1ms/step - loss: 0.5231 - acc: 0.9197
Epoch 6/10
598/598 [==============================] - 1s 2ms/step - loss: 0.4104 

In [ ]:
app = Flask(__name__)

@app.route("/testClass")
def index1():
    global model
    n = request.args.get("param")
    n = n[1:-1]
    n = [int(x) for x in n.split(",")]
    print(n)
    n = normalize(n)
    Xnew = [n]

    Xnew = sequence.pad_sequences(Xnew, maxlen=100,dtype='float32')
    print(Xnew)
    ynew = model.predict_classes(Xnew)
    print(ynew)
    return str(ynew[0][0])
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
